# Data Cleanup

## Setup

### Import Packages

In [84]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from collections import OrderedDict
from datetime import datetime, date
from os import environ
import json

# Set ipython's max row display
pd.set_option('display.max_row', 1000)
# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = [10, 5]

### Global Variables

## Importing Dataset

### Combined dataset

In [85]:
read_name = "../data/bronze_tables/combined_dataset.csv"

df_combined = pd.read_csv(read_name)

C:\Users\arahm\AppData\Local\Temp\ipykernel_27200\399399036.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined = pd.read_csv(read_name)


In [86]:
df_combined.head()

,Unnamed: 0,Entity_x,Code,Year,Annual CO2 emissions,Entity_y,Fossil fuels (% growth),Entity_x.1,Annual change in primary energy consumption (%),Entity_y.1,Gas Consumption - TWh,Coal Consumption - TWh,Oil Consumption - TWh,Entity_x.2,Fossil fuels (TWh),Entity_y.2,Coal Production - TWh,Oil Production - TWh,Gas Production - TWh,Entity_x.3,Fossil fuels per capita (kWh),Entity_y.3,Fossil fuels (% equivalent primary energy),Entity_x.4,"Gas (TWh, direct energy)","Oil (TWh, direct energy)","Coal (TWh, direct energy)",Entity_y.4,Geo Biomass Other - TWh,Solar Generation - TWh,Wind Generation - TWh,Hydro Generation - TWh,Entity_x.5,prod of Electricity from wind (TWh),prod of Electricity from hydro (TWh),prod of Electricity from solar (TWh),prod of Other renewables including bioenergy (TWh),Entity_y.5,Per capita electricity (kWh),Entity_x.6,Renewables per capita (kWh - equivalent),Entity_y.6,Renewables (% electricity)
0,0,Afghanistan,AFG,1949,14656.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Afghanistan,AFG,1950,84272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Afghanistan,AFG,1951,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Afghanistan,AFG,1952,91600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Afghanistan,AFG,1953,106256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
df_combined = df_combined[df_combined.columns[1:]]

In [144]:
df_combined["Country"].value_counts(dropna=False)

World                                                            271
United Kingdom                                                   271
Canada                                                           236
Germany                                                          229
Poland                                                           221
United States                                                    221
France                                                           213
Austria                                                          203
Belgium                                                          192
Spain                                                            191
Norway                                                           189
Hungary                                                          188
Sweden                                                           183
Denmark                                                          178
Netherlands                       

In [5]:
df = df_combined.copy()

#### Drop Duplicate Columns

In [6]:
cols = list(df.columns)
unique_cols = {}

for col in cols:
    unique_cols[col] = col.split('_', 1)[0]

In [7]:
df = df.rename(columns = unique_cols)

In [8]:
df = df.loc[:,~df.columns.duplicated()]

### Income Level Lookup table

In [10]:
read_name = "../data/bronze_tables/income_level_lookup.xlsx"

df_income_lookup = pd.read_excel(read_name, skiprows=0)

df_income_lookup.head()

,Economy,Code,Region,Income group,Lending category,Other (EMU or HIPC)
0,Aruba,ABW,Latin America & Caribbean,High income,NaN,NaN
1,Afghanistan,AFG,South Asia,Low income,IDA,HIPC
2,Angola,AGO,Sub-Saharan Africa,Lower middle income,IBRD,NaN
3,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,NaN
4,Andorra,AND,Europe & Central Asia,High income,NaN,NaN


In [11]:
df_income_lookup = df_income_lookup[df_income_lookup.columns[:5]]

#### join with combined_df

In [15]:
df_income_lookup = df_income_lookup.rename(columns={'Economy': 'Entity'})
df = df.set_index('Code').join(df_income_lookup.set_index('Code'), lsuffix="_" ).reset_index()

In [27]:
df.query("Year == 2020")[df["Income group"].isnull()]

C:\Users\arahm\AppData\Local\Temp\ipykernel_27200\2440540158.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.query("Year == 2020")[df["Income group"].isnull()]


,Code,Entity_,Year,Annual CO2 emissions,Fossil fuels (% growth),Annual change in primary energy consumption (%),Gas Consumption - TWh,Coal Consumption - TWh,Oil Consumption - TWh,Fossil fuels (TWh),Coal Production - TWh,Oil Production - TWh,Gas Production - TWh,Fossil fuels per capita (kWh),Fossil fuels (% equivalent primary energy),"Gas (TWh, direct energy)","Oil (TWh, direct energy)","Coal (TWh, direct energy)",Geo Biomass Other - TWh,Solar Generation - TWh,Wind Generation - TWh,Hydro Generation - TWh,prod of Electricity from wind (TWh),prod of Electricity from hydro (TWh),prod of Electricity from solar (TWh),prod of Other renewables including bioenergy (TWh),Per capita electricity (kWh),Renewables per capita (kWh - equivalent),Renewables (% electricity),Entity,Region,Income group,Lending category
268,AIA,Anguilla,2020,1.234260e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1895,BES,Bonaire Sint Eustatius and Saba,2020,3.006540e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4320,COK,Cook Islands,2020,7.942300e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.010000,0.000000,2346.041016,NaN,25.000000,NaN,NaN,NaN,NaN
6107,ESH,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
6818,FLK,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010000,0.000000,0.000000,0.000000,5313.496094,NaN,50.000000,NaN,NaN,NaN,NaN
7863,GLP,Guadeloupe,2020,2.561883e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.050000,0.030000,0.100000,0.360000,3993.438721,NaN,34.177216,NaN,NaN,NaN,NaN
8511,GUF,French Guiana,2020,7.283790e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.440000,0.050000,0.010000,3333.585449,NaN,51.546391,NaN,NaN,NaN,NaN
13472,MSR,Montserrat,2020,2.531800e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,2209.456543,NaN,0.000000,NaN,NaN,NaN,NaN
13543,MTQ,Martinique,2020,2.348393e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010000,0.000000,0.000000,0.130000,4130.680664,NaN,9.150327,NaN,NaN,NaN,NaN
13847,MYT,Mayotte,2020,3.055840e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Population & Area

In [28]:
read_name = "../data/bronze_tables/API_EN.POP.DNST_DS2_en_csv_v2_4701323.csv"

df_population = pd.read_csv(read_name, skiprows=4)

df_population.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,307.966667,312.411111,314.994444,316.827778,318.650000,320.566667,322.466667,324.316667,326.300000,328.166667,330.233333,332.494444,334.644444,336.261111,336.961111,336.588889,335.366667,333.900000,333.177778,333.872222,...,484.888889,494.494444,504.811111,516.066667,527.733333,538.977778,548.577778,555.711111,560.166667,562.366667,563.122222,563.622222,564.805556,566.944444,569.805556,573.138889,576.533333,579.661111,582.583333,585.338889,588.033333,590.611111,593.144444,NaN,NaN
1,Africa Eastern and Southern,AFE,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,9.206929,9.444024,9.690214,9.945378,10.209576,10.482895,10.765896,11.059617,11.365371,11.684171,12.016528,12.362473,12.721761,13.094008,13.478945,13.876693,14.287579,14.711372,15.148660,15.599232,...,25.947604,26.625607,27.321141,28.035481,28.769895,29.526360,30.001659,30.801413,31.628679,32.483559,33.367205,34.279582,35.219737,36.189915,36.879528,37.898051,38.940522,40.004465,41.089451,42.195162,43.319881,44.462045,45.620592,NaN,NaN
2,Afghanistan,AFG,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,14.058547,14.337645,14.631648,14.940699,15.265041,15.600080,15.945197,16.308762,16.702347,17.131463,17.594177,18.078319,18.565480,19.031569,19.455045,19.844369,20.194247,20.454746,20.561857,20.478206,...,30.261978,30.925972,31.859861,33.127872,34.651540,36.307546,37.910996,39.333171,40.527204,41.550591,42.503842,43.534959,44.747269,46.176059,47.776671,49.475786,51.164166,52.762987,54.249311,55.649251,56.992046,58.325678,59.684990,NaN,NaN
3,Africa Western and Central,AFW,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,10.877837,11.109940,11.351399,11.601453,11.859717,12.126330,12.402029,12.687791,12.984878,13.294421,13.616671,13.952068,14.302085,14.669124,15.053820,15.456176,15.876543,16.313802,16.766255,17.232596,...,28.002263,28.761516,29.540244,30.338334,31.157344,32.000505,32.872024,33.774930,34.710713,35.678698,36.677028,37.702723,38.753638,39.829118,40.929916,42.056948,43.211639,44.394917,45.607045,46.847251,48.114080,49.405535,50.720207,NaN,NaN
4,Angola,AGO,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,4.436874,4.498676,4.555554,4.600140,4.628678,4.637286,4.631780,4.629801,4.655231,4.724761,4.845784,5.012405,5.211585,5.423617,5.634069,5.839119,6.043005,6.249117,6.463553,6.690695,...,12.320205,12.727096,13.151101,13.592487,14.052633,14.535555,15.046232,15.588036,16.162593,16.768557,17.402451,18.059096,18.734457,19.427817,20.139508,20.867720,21.610470,22.366552,23.135062,23.916555,24.713072,25.527632,26.362612,NaN,NaN


In [39]:
read_name = "../data/bronze_tables/API_AG.LND.TOTL.K2_DS2_en_csv_v2_4701206.csv"

df_area = pd.read_csv(read_name, skiprows=4)

df_area.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,Land area (sq. km),AG.LND.TOTL.K2,NaN,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,180.0,...,180.0,180.0,180.0,180.0,180.0,180.0,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,180.00,1.800000e+02,1.800000e+02,180.00,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,NaN,NaN
1,Africa Eastern and Southern,AFE,Land area (sq. km),AG.LND.TOTL.K2,NaN,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,...,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,14571611.0,1.472019e+07,1.472024e+07,1.472023e+07,1.472027e+07,14720236.89,1.472027e+07,1.472096e+07,14721240.05,1.484517e+07,1.484513e+07,1.484509e+07,1.484514e+07,1.484515e+07,1.484514e+07,1.484515e+07,1.484516e+07,1.484512e+07,NaN,NaN
2,Afghanistan,AFG,Land area (sq. km),AG.LND.TOTL.K2,NaN,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,...,652230.0,652230.0,652230.0,652230.0,652230.0,652230.0,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,652230.00,6.522300e+05,6.522300e+05,652230.00,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,6.522300e+05,NaN,NaN
3,Africa Western and Central,AFW,Land area (sq. km),AG.LND.TOTL.K2,NaN,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046580.0,9046180.0,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,...,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,9045780.0,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9045780.00,9.045780e+06,9.045780e+06,9045780.00,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045780e+06,9.045773e+06,NaN,NaN
4,Angola,AGO,Land area (sq. km),AG.LND.TOTL.K2,NaN,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,...,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1246700.0,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1246700.00,1.246700e+06,1.246700e+06,1246700.00,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,1.246700e+06,NaN,NaN


In [40]:
df_pop_area = pd.concat([df_population, df_area])

df_pop_area = df_pop_area[df_population.columns[:-2]].drop(['Indicator Code'], axis=1)

In [41]:
df_pop_area.head(2)

,Country Name,Country Code,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,Population density (people per sq. km of land ...,NaN,307.966667,312.411111,314.994444,316.827778,318.650000,320.566667,322.466667,324.316667,326.300000,328.166667,330.233333,332.494444,334.644444,336.261111,336.961111,336.588889,335.366667,333.900000,333.177778,333.872222,336.450000,...,462.283333,474.722222,484.888889,494.494444,504.811111,516.066667,527.733333,538.977778,548.577778,555.711111,560.166667,562.366667,563.122222,563.622222,564.805556,566.944444,569.805556,573.138889,576.533333,579.661111,582.583333,585.338889,588.033333,590.611111,593.144444
1,Africa Eastern and Southern,AFE,Population density (people per sq. km of land ...,NaN,9.206929,9.444024,9.690214,9.945378,10.209576,10.482895,10.765896,11.059617,11.365371,11.684171,12.016528,12.362473,12.721761,13.094008,13.478945,13.876693,14.287579,14.711372,15.148660,15.599232,16.062631,...,24.633762,25.284822,25.947604,26.625607,27.321141,28.035481,28.769895,29.526360,30.001659,30.801413,31.628679,32.483559,33.367205,34.279582,35.219737,36.189915,36.879528,37.898051,38.940522,40.004465,41.089451,42.195162,43.319881,44.462045,45.620592


In [42]:
df_pop_area = df_pop_area.melt(
    id_vars=df_pop_area.columns[:3], 
    value_vars=df_pop_area.columns[3:],
    var_name='Year'
).reset_index(drop=True)

df_pop_area.head()

,Country Name,Country Code,Indicator Name,Year,value
0,Aruba,ABW,Population density (people per sq. km of land ...,1960,NaN
1,Africa Eastern and Southern,AFE,Population density (people per sq. km of land ...,1960,NaN
2,Afghanistan,AFG,Population density (people per sq. km of land ...,1960,NaN
3,Africa Western and Central,AFW,Population density (people per sq. km of land ...,1960,NaN
4,Angola,AGO,Population density (people per sq. km of land ...,1960,NaN


In [43]:
df_pop_area = df_pop_area.pivot(['Country Code', 'Year'], 'Indicator Name', 'value').reset_index()

C:\Users\arahm\AppData\Local\Temp\ipykernel_27200\2938234318.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df_pop_area = df_pop_area.pivot(['Country Code', 'Year'], 'Indicator Name', 'value').reset_index()


In [44]:
df_pop_area['Year'] = df_pop_area['Year'].astype(np.number)

#### join with combined_df

In [45]:
df_pop_area = df_pop_area.rename(columns={'Country Name': 'Entity'})

In [46]:
df_pop_area.head()

Indicator Name,Country Code,Year,Land area (sq. km),Population density (people per sq. km of land area)
0,ABW,1960.0,NaN,NaN
1,ABW,1961.0,180.0,307.966667
2,ABW,1962.0,180.0,312.411111
3,ABW,1963.0,180.0,314.994444
4,ABW,1964.0,180.0,316.827778


In [72]:
df_pop_area["Country Code"].value_counts(dropna=False)

ABW    61
OMN    61
MWI    61
MYS    61
NAC    61
NAM    61
NCL    61
NER    61
NGA    61
NIC    61
NLD    61
NOR    61
NPL    61
NRU    61
NZL    61
OED    61
OSS    61
PYF    61
PAK    61
PAN    61
PER    61
PHL    61
PLW    61
PNG    61
POL    61
PRE    61
PRI    61
PRK    61
PRT    61
PRY    61
PSE    61
PSS    61
MUS    61
MRT    61
MOZ    61
MNP    61
LDC    61
LIC    61
LIE    61
LKA    61
LMC    61
LMY    61
LSO    61
LTE    61
LTU    61
LUX    61
LVA    61
MAC    61
MAF    61
MAR    61
MCO    61
MDA    61
MDG    61
MDV    61
MEA    61
MEX    61
MHL    61
MIC    61
MKD    61
MLI    61
MLT    61
MMR    61
MNA    61
MNE    61
MNG    61
PST    61
QAT    61
AFE    61
UMC    61
TKM    61
TLA    61
TLS    61
TMN    61
TON    61
TSA    61
TSS    61
TTO    61
TUN    61
TUR    61
TUV    61
TZA    61
UGA    61
UKR    61
URY    61
ROU    61
USA    61
UZB    61
VCT    61
VEN    61
VGB    61
VIR    61
VNM    61
VUT    61
WLD    61
WSM    61
XKX    61
YEM    61
ZAF    61
ZMB    61
TJK    61


In [77]:
df = df.set_index(['Code', 'Year']).join(df_pop_area.set_index(['Country Code', 'Year'])).reset_index()

SyntaxError: invalid syntax (4085346474.py, line 1)

In [75]:
df["Code"].value_counts(dropna=False)

NaN         292751
GBR          16437
OWID_WRL     16437
CAN          16402
DEU          16395
POL          16387
USA          16387
FRA          16379
AUT          16369
BEL          16358
ESP          16357
NOR          16355
HUN          16354
SWE          16349
DNK          16344
NLD          16341
UKR          16335
UZB          16335
MDA          16335
TKM          16335
TJK          16335
RUS          16335
KAZ          16335
LVA          16335
LTU          16335
GEO          16335
EST          16335
BLR          16335
AZE          16335
ARM          16335
KGZ          16334
CHE          16329
ROU          16329
SVK          16327
ITA          16327
FIN          16327
CZE          16327
AUS          16327
TUR          16322
JPN          16319
IND          16318
PRT          16317
NZL          16309
GRC          16306
BGR          16306
ZAF          16303
PER          16303
ARG          16300
IDN          16298
MEX          16296
MYS          16296
VNM          16295
CHL         

### Rearrange Columns

In [48]:
cat_cols = ['Entity', 'Year', 'Code', 'Region', 'Income group', 'Lending category']
value_cols = list((set(df.columns) - set(cat_cols)) - {'Country Code'})

In [49]:
df = df[cat_cols + value_cols]

### Basic Analysis

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3841515 entries, 0 to 3841514
Data columns (total 35 columns):
 #   Column                                               Dtype  
---  ------                                               -----  
 0   Entity                                               object 
 1   Year                                                 float64
 2   Code                                                 object 
 3   Region                                               object 
 4   Income group                                         object 
 5   Lending category                                     object 
 6   prod of Electricity from solar (TWh)                 float64
 7   Coal Production - TWh                                float64
 8   Fossil fuels (% growth)                              float64
 9   Land area (sq. km)                                   float64
 10  Renewables (% electricity)                           float64
 11  Fossil fuels per capita 

In [51]:
df.describe()

,Year,prod of Electricity from solar (TWh),Coal Production - TWh,Fossil fuels (% growth),Land area (sq. km),Renewables (% electricity),Fossil fuels per capita (kWh),Geo Biomass Other - TWh,Oil Consumption - TWh,prod of Electricity from hydro (TWh),Oil Production - TWh,Renewables per capita (kWh - equivalent),Population density (people per sq. km of land area),Fossil fuels (TWh),prod of Other renewables including bioenergy (TWh),Annual change in primary energy consumption (%),"Oil (TWh, direct energy)",Gas Production - TWh,Annual CO2 emissions,Solar Generation - TWh,Per capita electricity (kWh),Coal Consumption - TWh,Wind Generation - TWh,Hydro Generation - TWh,Fossil fuels (% equivalent primary energy),"Coal (TWh, direct energy)","Gas (TWh, direct energy)",prod of Electricity from wind (TWh),Gas Consumption - TWh
count,3.841515e+06,1.866353e+06,362327.000000,1.110365e+06,3.692050e+06,1.481185e+06,1.115945e+06,1.103980e+06,1.122334e+06,1.884703e+06,665848.000000,1.122327e+06,3.451163e+06,1.115945e+06,1.849595e+06,2.466964e+06,15179.000000,655474.000000,3.717180e+06,1.103980e+06,1.524543e+06,1.115948e+06,1.103980e+06,1.122327e+06,1.115945e+06,15179.000000,15179.000000,1.866353e+06,1.122331e+06
mean,1.990628e+03,1.071659e+00,1896.652969,3.518550e+00,5.250017e+06,2.955008e+01,3.256567e+04,5.271232e+00,1.011397e+03,3.734946e+01,1702.987533,4.521244e+00,2.912407e+02,2.324555e+03,3.196876e+00,4.529973e+00,38418.751094,927.879854,4.702329e+08,1.801836e+00,3.889834e+03,7.492411e+02,5.355192e+00,6.128578e+01,8.641344e+01,28051.366649,20841.881473,3.180940e+00,5.562669e+02
std,1.818097e+01,1.794764e+01,6091.733428,2.724270e+01,1.509741e+07,3.208910e+01,3.447909e+04,3.296575e+01,4.682507e+03,2.362517e+02,5833.765701,1.334260e+01,1.501744e+03,1.096832e+04,2.559484e+01,3.188959e+01,9569.777345,3556.924536,2.167804e+09,2.330776e+01,5.077855e+03,3.740826e+03,5.427216e+01,3.037947e+02,1.555028e+01,9862.128852,9417.580614,4.182296e+01,2.738557e+03
min,1.750000e+03,0.000000e+00,0.339596,-4.959083e+01,2.027000e+00,0.000000e+00,1.558074e+02,0.000000e+00,9.738056e-02,0.000000e+00,0.000000,0.000000e+00,9.862452e-02,9.738055e-02,0.000000e+00,-9.500508e+01,0.000000,0.000000,3.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.280470e+01,97.000000,0.000000,0.000000e+00,0.000000e+00
25%,1.976000e+03,0.000000e+00,37.866039,-1.060772e+00,2.164000e+04,1.307330e+00,1.152530e+04,0.000000e+00,6.454946e+01,1.400000e-02,102.932060,1.683346e-01,1.946193e+01,1.164674e+02,0.000000e+00,-8.340657e-01,33660.101562,45.318611,6.885940e+05,0.000000e+00,5.642858e+02,3.265833e+00,0.000000e+00,4.206000e-01,8.080704e+01,19457.904297,13293.952148,0.000000e+00,8.645000e+00
50%,1.991000e+03,0.000000e+00,125.336014,2.466643e+00,1.998100e+05,1.599574e+01,2.540453e+04,3.900000e-02,1.425319e+02,1.520000e+00,344.388245,7.866168e-01,5.143897e+01,2.823127e+02,0.000000e+00,2.337921e+00,37960.613281,140.884018,6.938814e+06,0.000000e+00,2.427437e+03,3.254101e+01,0.000000e+00,4.582918e+00,9.176839e+01,25904.468750,20063.484375,0.000000e+00,5.259425e+01
75%,2.006000e+03,1.094000e-03,824.191345,6.335747e+00,1.260000e+06,5.312500e+01,4.025504e+04,1.299000e+00,4.147270e+02,9.815488e+00,1135.137939,2.784231e+00,1.282379e+02,9.501196e+02,3.500000e-01,6.949067e+00,46672.656250,399.114014,6.229507e+07,4.572000e-03,5.367686e+03,1.681976e+02,5.770781e-02,2.291953e+01,9.784651e+01,38069.035156,28161.105469,5.000000e-03,2.325982e+02
max,2.021000e+03,1.032501e+03,46550.605469,1.553105e+03,1.299870e+08,1.000000e+02,3.087042e+05,7.627827e+02,5.336863e+04,4.345990e+03,52181.949219,1.538834e+02,2.138860e+04,1.361315e+05,7.627827e+02,1.553105e+03,53368.628906,40368.828125,3.670250e+10,1.032501e+03,5.678160e+04,4.516121e+04,1.861940e+03,4.345990e+03,1.000000e+02,45161.207031,40374.605469,1.861940e+03,4.037461e+04


In [52]:
df.describe(include='object')

,Entity,Code,Region,Income group,Lending category,Entity_
count,3283267,3548764,3283267,3266991,2248104,3717180
unique,211,235,7,4,3,247
top,United Kingdom,GBR,Europe & Central Asia,High income,IBRD,United Kingdom
freq,16437,16437,838218,1155284,1119762,16436


### Data Clean-up

In [78]:
x = 'Entity'

#### Unify Entinty Names

In [79]:
df[x] = df[x].str.title()

In [80]:
df[x].value_counts()

United Kingdom                    16437
Canada                            16402
Germany                           16395
Poland                            16387
United States                     16387
France                            16379
Austria                           16369
Belgium                           16358
Spain                             16357
Norway                            16355
Hungary                           16354
Sweden                            16349
Denmark                           16344
Netherlands                       16341
Belarus                           16335
Tajikistan                        16335
Estonia                           16335
Uzbekistan                        16335
Ukraine                           16335
Turkmenistan                      16335
Russian Federation                16335
Moldova                           16335
Latvia                            16335
Lithuania                         16335
Kazakhstan                        16335


#### Create Entity Category Column

In [66]:
df[x].nunique()

247

In [67]:
continents = ['Asia', 'Africa', 'North America', 'South America', 'Europe', "Antarctica"]

countries_official_names = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']
other_countries = [
       'Antigua And Barbuda', 'Bolivia',
       'Bonaire Sint Eustatius And Saba', 'Bosnia And Herzegovina',
       'British Virgin Islands', 'Brunei', "Cote D'Ivoire", 'Curacao',
       'Czechia', 'Democratic Republic Of Congo', 'Eswatini',
       'Faeroe Islands', 'French Equatorial Africa', 'French West Africa',
       'Iran',
       'Kosovo',  'Laos', 'Leeward Islands',
       'Moldova',
       'North Korea', 'North Macedonia', 'Oceania', 'Palestine',
       'Panama Canal Zone', 'Reunion', 'Russia', 'Ryukyu Islands',
       'Saint Helena', 'Saint Kitts And Nevis',
       'Saint Pierre And Miquelon', 'Saint Vincent And The Grenadines',
       'Sao Tome And Principe', 'South Korea', 'St. Kitts-Nevis-Anguilla',
       'Syria', 'Taiwan', 'Tanzania', 'Timor', 'Trinidad And Tobago',
       'Turks And Caicos Islands',
       'Venezuela', 'Vietnam', 'Wallis And Futuna', 'Ussr',
       'Czechoslovakia', 'Falkland Islands', 'Netherlands Antilles',
       'Serbia And Montenegro', 'United States Virgin Islands',
       'Western Sahara', 'Yugoslavia',
       'Micronesia (Country)', 'Sint Maarten (Dutch Part)'
                  ]
countries = countries_official_names + other_countries

In [68]:
df[x]

0                United Kingdom
1                         World
2                        Europe
3          Europe (Excl. Eu-27)
4           European Union (28)
                   ...         
3841510                     NaN
3841511                     NaN
3841512                     NaN
3841513                     NaN
3841514                     NaN
Name: Entity_, Length: 3841515, dtype: object

In [69]:
def assign_category_to_entity(s):

    if "(" in s:
        if "(Bp)" in s:
            return "Bp?"
        if "(Eia)" in s:
            return "Eia?"
        if "(Ember)" in s:
            return "Ember?"
        if s in countries:
            return "Country"
        else:
            return "Exlusive Regions"

    else:
        if s == "World":
            return "World"
        elif s in continents:
            return "Continent"
        elif s in countries:
            return "Country"
        elif "Income" in s:
            return "Income Level"
        else:
            return "Other"

In [70]:
df['Entity_Category'] = df[x].apply(assign_category_to_entity)

TypeError: argument of type 'float' is not iterable

In [34]:
df['Entity_Category'].value_counts()

Country             21830
Bp?                  1549
Exlusive Regions     1368
Eia?                 1040
Continent             994
Income Level          725
World                 272
Ember?                110
Other                  72
Name: Entity_Category, dtype: int64

In [35]:
df_entity_lookup = df[['Entity_Category', 'Entity', 'Code', 'Income group', 'Lending category']].drop_duplicates().sort_values(by='Entity_Category')

In [36]:
df_entity_lookup['Entity_Category'].value_counts()

Country             240
Eia?                 29
Bp?                  28
Continent             6
Exlusive Regions      6
Ember?                5
Income Level          4
Other                 2
World                 1
Name: Entity_Category, dtype: int64

In [37]:
df_entity_lookup.head()

,Entity_Category,Entity,Code,Income group,Lending category
15505,Bp?,Middle East (Bp),NaN,NaN,NaN
15448,Bp?,Middle Africa (Bp),NaN,NaN,NaN
19236,Bp?,Other Middle East (Bp),NaN,NaN,NaN
19190,Bp?,Other Middle Africa (Bp),NaN,NaN,NaN
19133,Bp?,Other Europe (Bp),NaN,NaN,NaN


In [38]:
df.head()

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels (% equivalent primary energy),Fossil fuels (% growth),Gas Production - TWh,Land area (sq. km),Unnamed: 0.1.1.1,Coal Consumption - TWh,Renewables per capita (kWh - equivalent),Wind Generation - TWh,prod of Other renewables including bioenergy (TWh),Fossil fuels per capita (kWh),Fossil fuels (TWh),Oil Production - TWh,Coal Production - TWh,Hydro Generation - TWh,Unnamed: 0.1.1,prod of Electricity from solar (TWh),prod of Electricity from hydro (TWh),Unnamed: 0.1,"Gas (TWh, direct energy)",Annual change in primary energy consumption (%),Geo Biomass Other - TWh,"Coal (TWh, direct energy)",Per capita electricity (kWh),Oil Consumption - TWh,Renewables (% electricity),Population density (people per sq. km of land area),Solar Generation - TWh,Gas Consumption - TWh,"Oil (TWh, direct energy)",Annual CO2 emissions,prod of Electricity from wind (TWh),Entity_Category
0,Afghanistan,1949,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14656.0,NaN,Country
1,Afghanistan,1950,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84272.0,NaN,Country
2,Afghanistan,1951,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91600.0,NaN,Country
3,Afghanistan,1952,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91600.0,NaN,Country
4,Afghanistan,1953,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106256.0,NaN,Country


## Save Tables

In [39]:
df.head()

,Entity,Year,Code,Region,Income group,Lending category,Fossil fuels (% equivalent primary energy),Fossil fuels (% growth),Gas Production - TWh,Land area (sq. km),Unnamed: 0.1.1.1,Coal Consumption - TWh,Renewables per capita (kWh - equivalent),Wind Generation - TWh,prod of Other renewables including bioenergy (TWh),Fossil fuels per capita (kWh),Fossil fuels (TWh),Oil Production - TWh,Coal Production - TWh,Hydro Generation - TWh,Unnamed: 0.1.1,prod of Electricity from solar (TWh),prod of Electricity from hydro (TWh),Unnamed: 0.1,"Gas (TWh, direct energy)",Annual change in primary energy consumption (%),Geo Biomass Other - TWh,"Coal (TWh, direct energy)",Per capita electricity (kWh),Oil Consumption - TWh,Renewables (% electricity),Population density (people per sq. km of land area),Solar Generation - TWh,Gas Consumption - TWh,"Oil (TWh, direct energy)",Annual CO2 emissions,prod of Electricity from wind (TWh),Entity_Category
0,Afghanistan,1949,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14656.0,NaN,Country
1,Afghanistan,1950,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84272.0,NaN,Country
2,Afghanistan,1951,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91600.0,NaN,Country
3,Afghanistan,1952,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91600.0,NaN,Country
4,Afghanistan,1953,AFG,South Asia,Low income,IDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106256.0,NaN,Country


In [40]:
df.to_csv('../data/silver_tables/yearly_values_per_entity.csv', index=False)

In [41]:
df_entity_lookup.to_csv('../data/silver_tables/entity_lookup.csv', index=False)